## Lernhilfe für Lineare Regression

### 1. Datenbeschreibung

* Der Datensatz umfasst 30 Beobachtungen.
* Es gibt 6 Variablen:
    * **Species:** Anzahl der auf der Insel gefundenen Arten.
    * **Area:** Fläche der Insel (km²).
    * **Elevation:** Höchste Erhebung der Insel (m).
    * **Nearest:** Entfernung zur nächstgelegenen Insel (km).
    * **Scruz:** Entfernung zur Insel Santa Cruz (km).
    * **Adjacent:** Fläche der angrenzenden Insel (km²).

### 2. Pakete und Daten laden

Die Quellen enthalten keine spezifischen Informationen zum Laden von Paketen und Daten. Es wird davon ausgegangen, dass Sie die notwendigen Schritte zum Importieren der Daten und der benötigten Pakete (z.B. Pandas, NumPy, Statsmodels) in Ihrer bevorzugten Programmiersprache (z.B. Python, R) kennen.

### 3. Lineare Regression

#### 3.1. Regressionselemente extrahieren

##### 3.1.1. Grundlagen

Die Quellen enthalten keine detaillierten Informationen zu den grundlegenden Regressionselementen. Es wird davon ausgegangen, dass Sie mit Konzepten wie Koeffizienten, Standardfehlern, t-Werten und p-Werten vertraut sind.

##### 3.1.2. F-Werte

* F-Werte werden in Hypothesentests verwendet, um die Gesamtsignifikanz des Modells oder einer Gruppe von Prädiktoren zu bewerten.
* Die F-Statistik wird berechnet, indem die Varianz, die durch das Modell erklärt wird, durch die Restvarianz dividiert wird.
* Ein großer F-Wert deutet darauf hin, dass das Modell oder die Prädiktoren einen signifikanten Teil der Varianz in der Antwortvariablen erklären.

##### 3.1.3. Konfidenzintervalle

* Konfidenzintervalle liefern einen Bereich plausibler Werte für die Regressionskoeffizienten.
* Sie werden berechnet als Punktschätzung ± t-Wert * Standardfehler.
* Ein 95%-Konfidenzintervall bedeutet, dass wir zu 95% sicher sind, dass der wahre Wert des Koeffizienten innerhalb dieses Intervalls liegt.

##### 3.1.4. Anpassungsgüte

* Die Anpassungsgüte misst, wie gut das Modell die Daten anpasst.
* **R²:** Das Bestimmtheitsmaß ist eine gängige Metrik, die den Anteil der Varianz in der Antwortvariablen angibt, der durch das Modell erklärt wird.

##### 3.1.5. Quadratsummen

* Die Quadratsummen werden zur Berechnung verschiedener Statistiken in der linearen Regression verwendet, einschließlich der F-Statistik und des R².
* **TSS:** Die totale Quadratsumme misst die Gesamtvariation in der Antwortvariablen.
* **ESS:** Die erklärte Quadratsumme misst die Variation, die durch das Modell erklärt wird.
* **RSS:** Die Residuenquadratsumme misst die nicht erklärte Variation.

##### 3.1.6. Angepasste Werte und Residuen

* **Angepasste Werte:** Die vom Modell vorhergesagten Werte für die Antwortvariable.
* **Residuen:** Die Differenzen zwischen den beobachteten und den angepassten Werten.

##### 3.1.7. Kovarianzmatrix

Die Quellen enthalten keine spezifischen Informationen zur Kovarianzmatrix. Sie enthält Informationen über die Varianzen und Kovarianzen der geschätzten Regressionskoeffizienten.

##### 3.1.8. Ausreißertest

Die Quellen erwähnen Methoden zur Erkennung und Behandlung von Ausreißern, insbesondere im Kontext der robusten Regression.

#### 3.2. Schritt-für-Schritt-Berechnung der Schätzungen für Beta

##### 3.2.1. Verwendung der Moore-Penrose-Inversen zur Berechnung der Schätzungen

##### 3.2.2. Verwendung der QR-Zerlegung zur Berechnung der Schätzungen

##### 3.2.3. Verwendung des allgemeinen Lösers für das Problem der kleinsten Quadrate

Die Quellen enthalten keine detaillierten Informationen zur schrittweisen Berechnung der Schätzungen für Beta mithilfe dieser Methoden. Diese Methoden werden in fortgeschritteneren linearen Algebra- und numerischen Berechnungslehrbüchern behandelt.

#### 3.3. Identifizierbarkeit

Die Quellen definieren den Begriff "Identifizierbarkeit" nicht im Kontext der linearen Regression. Im Allgemeinen bezieht sich die Identifizierbarkeit auf die Fähigkeit, die Modellparameter eindeutig anhand der beobachteten Daten zu schätzen.

#### 3.4. Erklärung

* **Interpretation von Koeffizienten:** Eine Änderung um eine Einheit in $x_i$ bei Konstanz der anderen (genannten) Prädiktoren führt zu einer Änderung von $\hat{β_i}$ in der Antwortvariablen y.
* **Bedeutung der Konstanz:** Das Konzept, Variablen in Beobachtungsdaten konstant zu halten, ist nicht immer sinnvoll, da es möglicherweise nicht gemessene oder unbekannte Variablen gibt, die sich ebenfalls auf die Antwortvariable auswirken können.
* **Kausalität vs. Korrelation:** Regression impliziert keine Kausalität. Wir können zwar Vorhersagen treffen, aber wir können nicht behaupten, dass ein Prädiktor einen anderen verursacht.

### 4. Hypothesentests

#### 4.1. Test aller Prädiktoren

* **Nullhypothese:** Alle Regressionskoeffizienten sind gleich Null.
* **F-Test:** Verwendet, um die Nullhypothese zu testen.
* **Ablehnung der Nullhypothese:** Deutet darauf hin, dass mindestens ein Prädiktor signifikant mit der Antwortvariablen zusammenhängt.

#### 4.2. Test eines Prädiktors

* **Nullhypothese:** Der Regressionskoeffizient für den getesteten Prädiktor ist gleich Null.
* **t-Test:** Verwendet, um die Nullhypothese zu testen.
* **F-Test:** Kann auch verwendet werden, um einen einzelnen Prädiktor zu testen, und das Ergebnis ist äquivalent zum quadrierten t-Test.

#### 4.3. Test eines Prädiktorpaars

* **Nullhypothese:** Die Regressionskoeffizienten für beide getesteten Prädiktoren sind gleich Null.
* **F-Test:** Verwendet, um die Nullhypothese zu testen.

#### 4.4. Test eines Unterraums

* **Nullhypothese:** Eine lineare Kombination von Regressionskoeffizienten ist gleich einem bestimmten Wert.
* **F-Test:** Kann verwendet werden, um die Nullhypothese zu testen.

#### 4.5. Einschränkungen des Tests

* **Nichtlineare Hypothesen:** F-Tests können nicht verwendet werden, um nichtlineare Hypothesen zu testen.
* **Nicht verschachtelte Modelle:** F-Tests können nicht verwendet werden, um Modelle zu vergleichen, die nicht verschachtelt sind.
* **Unterschiedliche Datensätze:** F-Tests sind nicht direkt anwendbar, wenn die verglichenen Modelle unterschiedliche Datensätze verwenden.

#### 4.6. Permutationstests

* **Nichtparametrische Alternative:** Permutationstests sind eine Alternative zu F-Tests, die keine Normalverteilungsannahme erfordern.
* **Prinzip:** Die Antwortvariable wird permutiert, um die Verteilung der Teststatistik unter der Nullhypothese zu simulieren.
* **p-Wert:** Die Wahrscheinlichkeit, eine Teststatistik zu beobachten, die so extrem ist wie die beobachtete, unter der Annahme, dass die Nullhypothese wahr ist.
* **Anwendungen:** Kann verwendet werden, um alle Prädiktoren, einen einzelnen Prädiktor oder eine lineare Kombination von Prädiktoren zu testen.

## Lernhilfe für Konfidenzintervalle, Bootstrap, Diagnose und Transformation in der Regression

### 5. Konfidenzintervalle für $\beta$

* Die Formel für Konfidenzintervalle für $\beta$ ist: $\hat{β} *i \pm t* {\alpha/2, t-n} se(\hat{β})$
* **Konfidenzniveau und Hypothesentests:** Die Wahl eines bestimmten Konfidenzniveaus, z. B. 95%, bedeutet, dass Hypothesentests nur auf dem 5%-Signifikanzniveau durchgeführt werden können.
* **Breite des Konfidenzintervalls:** Ein breites Konfidenzintervall deutet auf eine geringere Sicherheit hinsichtlich des genauen Effekts des Prädiktors hin, selbst wenn statistische Signifikanz auf einen Zusammenhang hindeutet.
* **Vorteil von Konfidenzintervallen:** Sie bieten mehr Informationen über die möglichen Werte des Parameters als nur eine Punktschätzung und helfen, die Unsicherheit der Parameterschätzung besser zu verstehen.
* **Bessere Darstellung der Effektgröße:** Konfidenzintervalle eignen sich besser, um die Effektgröße zu verstehen.

#### Bootstrap-Konfidenzintervalle

* **Nichtparametrische Methode:** Bootstrap-Konfidenzintervalle ermöglichen die Erstellung von Konfidenzaussagen, ohne die Annahme einer Normalverteilung der Daten.
* **Resampling aus den Daten:** Bei der Bootstrap-Methode werden wiederholt Stichproben aus den beobachteten Daten gezogen, um mehrere Datensätze zu erstellen, die den Originaldaten ähneln.
* **Umgehung komplexer Berechnungen:** Die Methode ist auch dann effektiv, wenn theoretische Berechnungen schwierig oder unmöglich sind.
* **Schritte der Bootstrap-Methode:**
    1. Generiere $\bf{ε}^ *$ durch Ziehen mit Zurücklegen aus $\hat{ε}_1,\cdots,\hat{ε}_n$.
    2. Bilde $\bf{y}^* = \bf{X\hat{\beta}} + \bf{ε}^*$.
    3. Berechne $\bf{\hat{\beta}}$ aus $(\bf{X,~y^*})$.
* **Anzahl der Bootstrap-Stichproben:** Für eine Schätzung der Variabilität der Statistik genügen 50 Stichproben, für Konfidenzintervalle werden mehr benötigt.
* **Hypothesentests:** Bootstrap-Methoden können für Hypothesentests verwendet werden, Permutationstests sind jedoch in der Regel besser geeignet.
* **Alternative Resampling-Methoden:** Anstelle der Residuen können auch Paare von Variablen (X und Y) resampled werden. Dies ist jedoch möglicherweise weniger attraktiv, insbesondere wenn X als fest betrachtet wird, wie in geplanten Experimenten.

### 6. Diagnose

#### 6.1. Konstante Varianz

* **Varianzanalyse:** Es ist wichtig zu überprüfen, ob die Varianz der Residuen konstant ist (Homoskedastizität).
* **Transformationen:** Bei Zähldaten kann eine Quadratwurzeltransformation hilfreich sein, um die Varianz zu stabilisieren.
* **Poisson-Verteilung:** Die Poisson-Verteilung, die sich gut für Zähldaten eignet, hat einen gleichen Mittelwert und eine gleiche Varianz, was die Verwendung der Quadratwurzeltransformation unterstützt.

### 7. Robuste Regression

* **Problem mit Ausreißern:** Die Methode der kleinsten Quadrate (OLS) ist empfindlich gegenüber Ausreißern, die die Schätzungen der Regressionskoeffizienten verzerren können.
* **Robuste Regression:** Robuste Regressionsmethoden sind weniger anfällig für Ausreißer und liefern zuverlässigere Schätzungen.
* **M-Schätzung:** Eine Klasse von robusten Regressionsmethoden, die eine andere Verlustfunktion als OLS verwenden.
* **Wahl der Verlustfunktion:** Es gibt verschiedene Verlustfunktionen, die in der M-Schätzung verwendet werden können, z. B. die Huber-Funktion.
* **Iterative Berechnung:** Die Berechnung von M-Schätzungen erfordert iterative Schritte, da die Gewichte von den Residuen abhängen.
* **Standardfehler:** Standardfehler können durch gewichtete kleinste Quadrate (WLS) berechnet werden, wobei eine robuste Schätzung von $\sigma^2$ verwendet wird.
* **R²:** Die R²-Statistik ist im Kontext einer robusten Regression nicht sinnvoll.
* **Vergleich mit OLS:** Es ist hilfreich, die Ergebnisse der robusten Regression mit denen von OLS zu vergleichen, um festzustellen, ob es signifikante Unterschiede gibt.
* **Identifizierung problematischer Beobachtungen:** Robuste Regression kann helfen, Beobachtungen zu identifizieren, die vom Modell nicht gut angepasst werden.
* **Grenzen der robusten Regression:** Robuste Regression ist kein Allheilmittel und behebt nicht alle Probleme, die bei der Regression auftreten können, wie z. B. Punkte mit großem Einfluss oder die Auswahl von Prädiktoren.
* **LAD-Regression:** Eine weitere robuste Regressionsmethode, die den Betrag der Residuen minimiert.

### 8. Transformation

* **Transformation der Antwortvariablen:** Transformationen der Antwortvariablen können verwendet werden, um die Linearität, die Normalität der Residuen und die Homoskedastizität zu verbessern.
* **Log-Transformation:** Bei einer Log-Transformation der Antwortvariablen haben die Regressionskoeffizienten eine multiplikative Interpretation.
* **Box-Cox-Methode:** Eine allgemeine Methode zur Auswahl einer geeigneten Transformation der Antwortvariablen.
* **Wahl von λ:** Der Parameter λ in der Box-Cox-Transformation wird durch Maximierung der Profil-Log-Likelihood ausgewählt.
* **Interpretation der transformierten Antwortvariablen:** Die transformierte Antwortvariable kann schwieriger zu interpretieren sein als die ursprüngliche Variable.
* **Runden von λ:** Um die Interpretation zu vereinfachen, kann λ auf einen interpretierbareren Wert gerundet werden.
* **Konfidenzintervall für λ:** Ein Konfidenzintervall für λ kann verwendet werden, um zu beurteilen, wie sinnvoll eine Rundung von λ ist.
* **Notwendigkeit der Transformation:** Transformationen sollten nur durchgeführt werden, wenn sie notwendig sind, da sie das Modell komplexer machen.